#1.Setup - Clone Repo
Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
import os
%cd /content

!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

##Load dataset from Roboflow (Export it using YOLOv5 Pytorch Format)

In [ ]:
### load dataset
!curl -L "https://public.roboflow.com/ds/QNtFqEfj6m?key=Jrb6UojIIh" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

##Run it to customize files

In [ ]:
#customize iPython writefile so you can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

#Image Visualisation

In [ ]:
def get_all_images(folder, ext):

    all_files = []
    #Iterate through all files in folder
    for file in os.listdir(folder):
        #Get the file extension
        _,  file_ext = os.path.splitext(file)

        #If file is of given extension, get it's full path and append to list
        if ext in file_ext:
            full_file_path = os.path.join(folder, file)
            all_files.append(full_file_path)

    #Get list of all files
    return all_files

data_path = "/content/yolov5/valid/images"

#List of all jps and tif files
files = get_all_images(data_path, 'jpg')
print(len(files))


In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
for img in files:
 image = cv2.imread(img)
 image = image.astype("float32")
 #image = np.true_divide(image, 1, dtype=np.float16)
#print(image.dtype)
#print(image)
cv2_imshow(image)

# Customize YAML file (This step is not needed if you use Roboflow to load the dataset)

## Modify data file

In [ ]:
%cat data.yaml
# define number of classes based on YAML
# data.yaml contains the information about number of classes and their labels required for this project
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
    #customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))


# Train

Train a YOLOv5s model on your custom dataset by specifying:  batch-size = 32, image size = 320, epochs = 120 ( it depends on the dataset size) and either pretrained --weights yolov5s.pt (recommended), or randomly initialized --weights '' --cfg yolov5s.yaml (not recommended).

## Load Pre-trained weights (In case of using it instead of YAML file (yolov5s.yaml)

In [ ]:
!pwd
%cd /content/yolov5
!wget https://github.com/ultralytics/YOLOv5/releases/download/v6.1/yolov5s.pt

## Start training

In [ ]:
%%time
!python train.py --batch 16 --epochs 50 --data /content/yolov5/data.yaml  --weights '/content/yolov5/yolov5s.pt' --cache #--cfg /content/yolov5/models/yolov5s.yaml

## Validation & Model evaluation

In [ ]:
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/yolov5/data.yaml

## Run detection on a testing image

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --source /content/yolov5/valid/images/IMG_8348_jpg.rf.f98cd0dfc3a2c7693b77fb490d1405d1.jpg